In [1]:
import main
main.create_csv_from_las()

Convert224ST4.LAS
Convert238ST4.LAS


Convert274ST4.LAS
Convert300.LAS


Convert301.LAS
Convert302.LAS


Convert304.LAS


Convert305.LAS
Convert306.LAS


...

Convert841.LAS
Convert850.LAS


Convert852.LAS
Convert853.LAS


Convert855.LAS
Convert856.LAS


Convert857.LAS
Convert859.LAS


end save csv


In [150]:
import numpy as np
import pandas as pd
import main
las_data = pd.read_csv(main.csv_out_file, delimiter=';')
# пока nan заменяем на 0, потом надо другое что-то сделать
las_data = las_data.fillna(0)

В следующем блоке выделяем данные из базы для последующего обучения. Данные преобразуем к следующему виду: скважины - строки, усредненные значения по параметру с соответсвующим шагом (столбцы), количество разбиений по глубине зависит от параметра <b>step_h_count</b>

In [127]:
data_union = pd.DataFrame() 
well_name_list = las_data['well_name'].value_counts().index.tolist()
step_h_count = 5
well_i = 0
for well_name in well_name_list:
    data_well_list = []
    # Выделяем даные для одной скважины
    data_well = las_data[las_data['well_name'] == well_name]

    # высчитываем для нее характерисктики шага по глубине
    start_h = data_well[:1]["STRT"]
    end_h = data_well[:1]["STOP"]
    start_h = float(start_h)
    end_h = float(end_h)
    h_step = (float(end_h) - float(start_h))/step_h_count
    #print(float(start_h), float(end_h), h_step)
    temp_start_h = start_h
    index = 1
    while temp_start_h < end_h:
        str_index = str(index)
        temp_end_h = temp_start_h + h_step
        # выделяем для конкретного участка по глубине
        data_well_by_h = data_well[(data_well['DEPT'] >= temp_start_h) & (data_well['DEPT'] < temp_end_h)]
        columns_to_show=['aps', 'rp', 'kp', 'kgl', 'kpr', 'kvo', 'kng', 'lit', 'satur', 'wc']
        # расчитываем среднее
        data_well_by_h = data_well_by_h.pivot_table(['aps', 'rp', 'kp', 'kgl', 'kpr', 'kvo', 'kng', 'lit', 'satur'], ['well_name'] ,aggfunc='mean')
        data_well_by_h = data_well_by_h.rename(index=str, columns={"aps": 'aps' + str_index, 'rp': 'rp' + str_index, 'kp': 'kp' + str_index, 'kgl': 'kgl' + str_index, 
                                                                'kpr': 'kpr' + str_index, 'kvo': 'kvo' + str_index, 'kng': 'kng' + str_index, 'lit': 'lit'  + str_index,
                                                                'satur': 'satur' + str_index})
        data_well_list.append(data_well_by_h)
        temp_start_h = temp_end_h
        index = index + 1
        # print(temp_start_h, temp_end_h, end_h)
    # делаем ключи и праметры плоскими (размерности 1)
    key_list = list([])
    for elem in data_well_list:
        key_list.extend(list(elem))
    values_list = np.vstack(data_well_list)
    values_list = values_list.ravel()
    values_list = values_list.tolist()
    values_data = [values_list]
    
    # Создаем плоский срез для наших данных
    temp_frame = pd.DataFrame(data = values_data,
                 columns=key_list, index = [well_i])
    temp_frame['well_name'] = well_name
    
    
    if well_i == 0:
        data_union = temp_frame
    else:
        data_union = data_union.append(temp_frame)
    
    well_i = well_i + 1
    # print('temp_frame:', temp_frame)
    # if well_i == 3:
    #break
print("End create data_union")

End create data_union


In [184]:
#data_for_analize[np.isnan((data_for_analize["aps1"])==True)]
# для удобства перенесем столбец well_name в начало
# добавляем wc
water_content_data = pd.read_csv(main.data_dir + "\\res.csv", delimiter=';')
water_content_data = water_content_data.rename(columns={'Скважина': main.keys_dict[main.kid_well]})
water_content_data = water_content_data.rename(columns={' Начальная обводненность ': 'wc'})
del water_content_data['Дата запуска']
data_for_analize = pd.merge(data_union, water_content_data, on=main.keys_dict[main.kid_well])

# разобраться почему есть Nan
data_for_analize = data_for_analize.fillna(0)
y = data_for_analize['wc']
well_names = data_for_analize['well_name']
data_for_analize.drop(['wc', 'well_name'], axis=1, inplace=True)

In [202]:
y_r = int(10) * round(y,1)
y_r = list(map(int, y_r))
y_r

[5,
 1,
 3,
 4,
 3,
 3,
 4,
 2,
 4,
 3,
 3,
 2,
 3,
 2,
 2,
 3,
 7,
 4,
 2,
 5,
 3,
 4,
 2,
 3,
 5,
 5,
 1,
 1,
 2,
 1,
 4,
 3,
 2,
 2,
 3,
 4,
 2,
 3,
 7,
 1,
 4,
 2,
 2,
 4,
 4,
 7,
 1,
 2,
 2,
 4,
 6,
 4,
 2,
 1,
 4,
 4,
 5,
 4,
 2,
 5,
 3,
 4,
 2,
 2,
 2,
 3,
 3,
 8,
 3,
 4,
 4,
 3,
 2,
 1,
 2,
 4,
 2,
 1,
 2,
 4,
 4,
 4,
 4,
 4,
 2,
 6,
 1,
 3,
 1,
 3,
 4,
 5,
 2,
 4,
 1,
 5,
 2,
 3,
 4,
 6,
 4,
 4,
 5,
 1,
 2,
 2,
 2,
 6,
 3,
 6,
 4,
 1,
 2,
 2,
 1,
 7,
 9,
 2,
 3,
 2,
 1,
 4,
 2,
 5,
 2,
 2,
 4,
 4,
 3,
 3,
 5,
 2,
 2,
 3,
 5,
 3,
 5,
 5,
 2,
 5,
 5,
 5,
 4,
 8,
 4,
 8,
 3,
 4,
 3,
 2,
 2,
 4,
 0,
 3,
 3,
 3,
 0,
 2,
 5,
 1,
 2,
 0,
 1,
 2,
 3,
 2,
 1,
 1,
 6,
 1,
 1,
 1,
 1,
 3]

In [224]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.cross_validation import cross_val_score

x_train, x_hold, y_train, y_hold = train_test_split(data_for_analize.values, y_r, test_size=0.3) #random_state=17)
tree = DecisionTreeClassifier(max_depth=20, random_state=17)
tree.fit(x_train, y_train)
tree_pred = tree.predict(x_hold)
accuracy_score(y_hold, tree_pred)


#regressor = DecisionTreeRegressor(max_depth=10)
#regressor.fit(x_train, y_train)
#ree_pred = regressor.predict(x_hold)
#scores = cross_val_score(regressor, data_for_analize.values, y, cv=5, scoring='r2')
#print("R2-Score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

#dt_old = DecisionTreeRegressor(min_samax_depth=20, random_state=99)
#dt_old.fit(x_train, y_train)
# scores = cross_val_score(dt_old, x_hold, y_hold, cv=5)
# print("mean: {:.3f} (std: {:.3f})".format(scores.mean(),
#                                           scores.std()),
#                                           end="\n\n" )

0.24528301886792453